# Script for Updating group_RT for fmri glm

## assumptions
#### 1) only participants that will be modeled are in BIDS_scans

#### 2) the BIDS_dir is locally accessible (either by having the behavioral data in BIDS format locally, or via mount). Could be run on sherlock with minor changes.

In [1]:
import numpy as np
import pandas as pd
from os import path
from glob import glob
import sys

In [ ]:
aim = 'aim1'

In [15]:
if aim == 1:
    BIDS_mnt = '/Users/henrymj/Documents/SRO/tmp/OAK/data/uh2/aim1/BIDS_scans' #path to sherlock BIDS_scans, mounted to my local
elif aim == 2:
    BIDS_mnt = '/Users/henrymj/Documents/SRO/tmp/OAK/data/uh2/aim2/BIDS_scans' #path to sherlock BIDS_scans, mounted to my local

In [24]:
events_files = []
if aim == 1:
    events_files = glob(path.join(BIDS_mnt, 'sub-s*', 'ses-*', 'func', '*events.tsv')) #grab all of the events files - AIM1
elif aim == 2:
    events_files = glob(path.join(BIDS_mnt, 'sub-*', 'func', '*events.tsv')) #grab all of the events files - AIM2

events_files.sort()

In [25]:
tasks = 'ANT CCTHot DPX discountFix manipulationTask motorSelectiveStop stopSignal stroop twoByTwo WATT3'.split(' ') #I'm too lazy to write the list

In [27]:
def get_stop_RTs(long_df):
    go_long = long_df[long_df.trial_type.isin(['noncrit_nosignal', 'noncrit_signal','crit_go', 'go'])]
    go_groupRT = np.mean(go_long[go_long.junk != True].response_time)

    stop_long = long_df[long_df.trial_type.isin(['crit_stop_failure', 'stop_failure'])]
    failedStop_groupRT = np.mean(stop_long[stop_long.junk != True].response_time)
    
    return (go_groupRT, failedStop_groupRT)

## part 1: get group_RTs

In [28]:

group_RTs = {}
for task in tasks:
    curr_events = [i for i in events_files if task in i]
    curr_events.sort()
    
    long_df = None
    for subj_file in curr_events:
        tmp_df = pd.read_csv(subj_file, delimiter='\t')
        if long_df is None:
            long_df = tmp_df
        else:
            long_df = long_df.append(tmp_df)
    if long_df is not None: #skipping manipulation for aim1
        if task in 'stopSignal motorSelectiveStop'.split(' '):
            curr_goRT, curr_failedStopRT = get_stop_RTs(long_df)
            group_RTs[task] = (curr_goRT, curr_failedStopRT) #add to dict
        else:
            curr_gRT = np.mean(long_df[long_df.junk==False].response_time)
            group_RTs[task] = curr_gRT #add to dict
            
group_RTs

{'discountFix': 1.4916855420663926,
 'manipulationTask': 0.7659885377942974,
 'motorSelectiveStop': (0.6704018421928775, 0.6363366430260062),
 'stopSignal': (0.7314645946342206, 0.6184035006909255)}

## Part 2: insert group_RTs

In [29]:
#part 2: use group_RTs
for task in tasks:
    print(task)
    curr_events = [i for i in events_files if task in i]
    curr_events.sort()
    print('num events files: ' + str(len(curr_events )))
    
    for subj_file in curr_events:
        tmp_df = pd.read_csv(subj_file, delimiter='\t')
        if task in 'stopSignal motorSelectiveStop'.split(' '):
            tmp_df['group_RT'] = 0.0 #initialize column
            #add go RTs
            tmp_df['group_RT'] = tmp_df.apply(lambda x: group_RTs[task][0] if (x['trial_type'] in ['noncrit_nosignal', 'noncrit_signal','crit_go', 'go']) else x['group_RT'], axis=1)
            #add stop_failure_RTs
            tmp_df['group_RT'] = tmp_df.apply(lambda x: group_RTs[task][1] if (x['trial_type'] in ['crit_stop_failure', 'stop_failure']) else x['group_RT'], axis=1)
        else:
            tmp_df['group_RT'] = group_RTs[task] #insert group_RT
            
        tmp_df.to_csv(subj_file, sep='\t', index=False) #save df w/ group_RT

In [9]:
group_RTs

{'ANT': 0.602774296874999,
 'CCTHot': 1.605532885359399,
 'DPX': 0.5214033843149996,
 'discountFix': 1.4133896354779107,
 'motorSelectiveStop': (0.6444886092989532, 0.6378897389738966),
 'stopSignal': (0.6686504907306444, 0.5847133220910642),
 'stroop': 0.6869499352611136,
 'twoByTwo': 0.7453855379343609,
 'WATT3': 1.0660380757555799}